In [1]:
# Text Classification / tc-reuter.ipynb
# Gourav Siddhad
# 07-Mar-2019

In [34]:
print('Importing Libraries', end='')

import nltk
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer

import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

import matplotlib.pyplot as plt
from scipy import interp
from itertools import cycle

from bs4 import BeautifulSoup
import string
import csv
import os
import itertools
import tarfile
import re
import numpy as np
import time

print(' - Done')

Importing Libraries - Done


# Read Data

In [44]:
if not os.path.exists('reuters-data'):
    os.mkdir('reuters-data')

# Decompress data
# compressed = tarfile.open('reuters-data/reuters21578.tar.gz')
# compressed.extractall(path = 'reuters-data/')
# compressed.close()

# List the uncompressed data
print('Directory Listing')
os.listdir('reuters-data/')

Directory Listing


['all-exchanges-strings.lc.txt',
 'all-orgs-strings.lc.txt',
 'all-people-strings.lc.txt',
 'all-places-strings.lc.txt',
 'all-topics-strings.lc.txt',
 'cat-descriptions_120396.txt',
 'feldman-cia-worldfactbook-data.txt',
 'lewis.dtd',
 'README.txt',
 'reut2-000.sgm',
 'reut2-001.sgm',
 'reut2-002.sgm',
 'reut2-003.sgm',
 'reut2-004.sgm',
 'reut2-005.sgm',
 'reut2-006.sgm',
 'reut2-007.sgm',
 'reut2-008.sgm',
 'reut2-009.sgm',
 'reut2-010.sgm',
 'reut2-011.sgm',
 'reut2-012.sgm',
 'reut2-013.sgm',
 'reut2-014.sgm',
 'reut2-015.sgm',
 'reut2-016.sgm',
 'reut2-018.sgm',
 'reut2-019.sgm',
 'reut2-020.sgm',
 'reut2-021.sgm']

In [45]:
# Load in each data file
text_data = []

print('Loading Files - ', end=' ')
for index in range(22):
    if index is not 17:
        print(index, end=' ')
        filename = 'reuters-data/reut2-{0}.sgm'.format(str(index).zfill(3))
        with open(filename, 'r', encoding = 'utf-8', errors = 'ignore') as infile:
            text_data.append(infile.read())

print()
print('Loading Files - Done')

Loading Files -  0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 18 19 20 21 
Loading Files - Done


In [69]:
import pandas as pd

# Separate each text file into articles

print('Converting Files to Articles - ', end='')
i, counter = 0, 0

columns = ['Title', 'Topics', 'Text']
index = [x for x in range(0, 18103)]
df = pd.DataFrame(columns=columns, index=index)

for textfile in text_data:
    print(i, end=' ')
    # Parse text as html
    soup = BeautifulSoup(textfile, 'html.parser')
    
    # Extract article between <BODY> and </BODY> and convert to standard text. Add to list of articles
    title = soup.find('reuters')['lewissplit']
    topics = soup.find_all('topics')
    
    j=0
    for article in soup.find_all('body'):
        df['Text'][counter] = article.get_text()
        df['Title'][counter] = title
        df['Topics'][counter] = topics[j].find_all('d')
        j += 1
        counter += 1
    
    i += 1

print()
print('Files to Aritcle Conversion - Done')
print('Total Articles - ', counter)

Converting Files to Articles - 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
Files to Aritcle Conversion - Done
Total Articles -  18103


In [79]:
df

,Title,Topics,Text
0,TRAIN,[<d>cocoa</d>],Showers continued throughout the week in\nthe ...
1,TRAIN,[],Standard Oil Co and BP North America\nInc said...
2,TRAIN,[],Texas Commerce Bancshares Inc's Texas\nCommerc...
3,TRAIN,[],BankAmerica Corp is not under\npressure to act...
4,TRAIN,"[<d>grain</d>, <d>wheat</d>, <d>corn</d>, <d>b...",The U.S. Agriculture Department\nreported the ...
5,TRAIN,"[<d>veg-oil</d>, <d>linseed</d>, <d>lin-oil</d...",Argentine grain board figures show\ncrop regis...
6,TRAIN,[],Red Lion Inns Limited Partnership\nsaid it fil...
7,TRAIN,[],Moody's Investors Service Inc said it\nlowered...
8,TRAIN,[<d>earn</d>],Champion Products Inc said its\nboard of direc...
9,TRAIN,[<d>acq</d>],Computer Terminal Systems Inc said\nit has com...


In [78]:
count = 0
for topic in df['Topics']:
    if len(topic)>0:
        count += 1
print('Documents with Complete Details - ', count)

Documents with Complete Details -  9626
